In [1]:
%load_ext autoreload

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from src.datalib import *
from scipy.ndimage import gaussian_filter
from scipy import ndimage
import os
import pandas

from tqdm import tqdm
%matplotlib widget

In [2]:
wavelength_to_index = lambda x, wavelengths: np.argmin( np.abs( wavelengths - x ) )

def min_max(arr):
    return  abs( (arr - arr.min()) / (arr.max() - arr.min()) )#.astype(np.uint8)

# 1. Load LIBS Dataset

In [3]:
file_libs_metadata = r'E:/Data/Data_LIBS/Data_Brno/P36 - Epithelioid angiosarcoma/LIBS/Kopie souboru 403x540-P36B-Avantes.libsmetadata.txt'
file_libs_data = r"E:/Data/Data_LIBS/Data_Brno/P36 - Epithelioid angiosarcoma/LIBS/Kopie souboru 403x540-P36B-Avantes.libsdata"

libs_manipulator = MapData(file_path = file_libs_data)
libs_manipulator.load_wavelenths()
libs_manipulator.load_all_data('libs')
libs_manipulator.baseline_correct()

loading wavelengths
loading metadata
preprocessed file was not found; setting overwrite to True
loading raw data
getting baselines


In [ ]:
libs_manipulator.get_map_dimensions()
nx, ny = libs_manipulator.map_dimensions
spectrum = libs_manipulator.spectra.reshape(ny, nx, -1)
wavelengths = libs_manipulator.wvl

# Correct flipped dimensions
spectrum[1::2, :] = spectrum[1::2, ::-1]

getting map dimensions


# 2. Load ICP-MS Data

In [ ]:
main_dir = r""
sample_dir = r"P36 - Epithelioid angiosarcoma//"
dir_ICP = main_dir + sample_dir + r"LA-ICP-MS//P36B//Export//"
dir_image = main_dir + sample_dir + r"Histology//"
dir_LIBS = main_dir + sample_dir + r'LIBS//'


def read_image(folder,index=0):
    files = [folder+'//'+f for f in os.listdir(folder) if f.endswith('jpg')]
    return skimage.io.imread(files[index])

def read_metadata(file1):
    df = pandas.read_csv(file1,encoding = "ISO-8859-1",header=None)
    map_dims = [float(df[1][0].split('x')[0]),float(df[1][0].split('x')[1])]
    spot_dims = [float(df[1][1].split('x')[0]),float(df[1][1].split('x')[1])]
    return map_dims, spot_dims

def read_icp(folder, name='Total'):
    file1 = [folder+'//'+f for f in os.listdir(folder) if f.endswith('csv') and 'metadata' in f]
    print(file1)
    map_dims, spot_dims = read_metadata(file1[0])
    files = [folder+'//'+f for f in os.listdir(folder) if f.endswith('csv') and name in f]
    df = pandas.read_csv(files[0],encoding = "ISO-8859-1",header=None)
    df = df.fillna(0)
    data = df.to_numpy()
    data-=data.min()+1
    return map_dims, spot_dims, data

names_ICP = ['Mg','Cu','Fe']
n_ICP = len(names_ICP)
data_ICP = []
for i in range(0, n_ICP):
    map_dims, spot_dims, data = read_icp(dir_ICP,name=names_ICP[i])
    data_ICP.append(data)

fig1, ax = plt.subplots(1,1+n_ICP,figsize=[10,5])

sample_image = read_image(dir_image)
ax[0].imshow(sample_image,origin='lower')

for i in range(0, n_ICP):
    ax[i+1].imshow(data_ICP[i], norm = colors.LogNorm(vmin = 10, vmax=data.max()), cmap = 'cividis', extent=[0,map_dims[0],0,map_dims[1]])
    ax[i+1].set_xlabel(r'$x (\mu m)$')
    ax[i+1].set_ylabel(r'$y (\mu m)$')
    ax[i+1].set_title('LA-ICP-MS '+ str(names_ICP[i]))